# P1: Solve the OpenAI Gym [Taxi V3](https://gym.openai.com/envs/Taxi-v3/) Environment
---

## Introduction
[OpenAI Gym](https://gym.openai.com/docs/) is a framework that provides RL environments of varying complexity with the same standard API making it easy to develop and benchmark RL algorithms. The [Taxi-V3](https://gym.openai.com/envs/Taxi-v3/) environmnet present a simple, text environment where actions and state (observations) are both discrete. 

In [1]:
import gym

The `gym.make()` API can be used to spawn any of the available environments by passing its full name.

In [2]:
taxi = gym.make('Taxi-v3')

The Taxi environment has 500 states and 6 possible actions.

In [3]:
taxi.action_space

Discrete(6)

In [4]:
taxi.observation_space

Discrete(500)

The task and reward structure are described in the [documentation](https://github.com/openai/gym/blob/a5a6ae6bc0a5cfc0ff1ce9be723d59593c165022/gym/envs/toy_text/taxi.py#L25)

In [5]:
taxi.reset()
taxi.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [6]:
HYPER_PARAMS = {'gamma':0.9}

In [7]:
def epsilon_greedy_action_from_Q(Q, state, epsilon):
    actions = Q.columns
    action_probs = np.asarray([epsilon/len(actions)]*len(actions),dtype=np.float)
    
    greedy_action_index = np.argmax(Q.loc[state].values)
    action_probs[greedy_action_index] += 1-epsilon

    epsilon_greedy_action = np.random.choice(Q.columns,p=action_probs)
    
    return epsilon_greedy_action

In [14]:
def greedy_policy_from_returns_tbl(table):
    policy = {s:None for s in table.index }
    for state in table.index:        
       greedy_action = table.loc[state].idxmax()
       policy[state] = greedy_action
            
    return policy

In [16]:
def pretty_print_policy(policy, drop_off=False, render=True):
    if render:
      taxi.render()
    taxi_row, taxi_col, pass_idx, dest_idx = taxi.decode(taxi.s)
    if drop_off: pass_idx = 4
    direction_repr = {1:' 🡑 ', 2:' 🡒 ', 3:' 🡐 ', 0:' 🡓 ', None:' ⬤ ', 4:' O ', 5:' X '}
    for row in range(5):
        for col in range(5):
            state = taxi.encode(row, col, pass_idx, dest_idx)
            print(direction_repr[policy[state]],end='')
        print()

In [19]:
from tqdm import tqdm 
import numpy as np
import pandas as pd
Q = pd.DataFrame.from_dict({s:{a:0 for a in range(taxi.nA)} for s in range(taxi.nS)}, orient='index')

n_episodes = 5000
epsilon = 1
min_epsilon = 0.1
epsilon_decay = 0.9995

alpha = 0.0001

rewards = np.zeros(n_episodes)


for i in tqdm(range(n_episodes)):
    taxi.reset()
    s0 = taxi.s
    done = False
    
    episode_reward = 0
    while not done:
        a0 = epsilon_greedy_action_from_Q(Q,s0,epsilon)
        out  = taxi.step(a0)
        s1 = out[0]
        reward = out[1]
        done = out[2]
        
        Q.loc[s0,a0] += alpha*(reward + HYPER_PARAMS['gamma']*Q.loc[s1].max() - Q.loc[s0,a0])
        episode_reward += reward
        s0 = s1
  
    epsilon *= epsilon_decay
    epsilon = max(epsilon,min_epsilon) if i<n_episodes-1000 else 0  

    rewards[i] = episode_reward
    if (i+1)%500 == 0:
        print(f'Iteration {i+1}')
        policy = greedy_policy_from_returns_tbl(Q)
        pretty_print_policy(policy)
        

policy_Q_Learning = greedy_policy_from_returns_tbl(Q)
print(policy_Q_Learning)

pretty_print_policy(policy_Q_Learning)



  0%|          | 0/5000 [00:00<?, ?it/s]

  0%|          | 1/5000 [00:00<11:04,  7.52it/s]

  0%|          | 2/5000 [00:00<11:44,  7.09it/s]

  0%|          | 3/5000 [00:00<12:50,  6.48it/s]

  0%|          | 4/5000 [00:00<13:38,  6.11it/s]

  0%|          | 5/5000 [00:00<13:12,  6.31it/s]

  0%|          | 6/5000 [00:00<12:59,  6.41it/s]

  0%|          | 7/5000 [00:01<12:56,  6.43it/s]

  0%|          | 8/5000 [00:01<13:26,  6.19it/s]

  0%|          | 9/5000 [00:01<13:49,  6.02it/s]

  0%|          | 10/5000 [00:01<13:49,  6.02it/s]

  0%|          | 11/5000 [00:01<14:01,  5.93it/s]

  0%|          | 12/5000 [00:01<13:46,  6.04it/s]

  0%|          | 13/5000 [00:02<13:16,  6.26it/s]

  0%|          | 14/5000 [00:02<13:07,  6.33it/s]

  0%|          | 15/5000 [00:02<12:44,  6.52it/s]

  0%|          | 16/5000 [00:02<13:11,  6.29it/s]

  0%|          | 17/5000 [00:02<13:40,  6.07it/s]

  0%|          | 18/5000 [00:02<14:10,  5.86it/s]

  0%|          | 19/5000 [00:03<14:17,  5.81it/

Iteration 500
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
 🡑  🡒  🡐  🡐  🡓 
 🡓  🡒  🡑  🡓  🡑 
 🡐  🡐  🡐  🡒  🡒 
 🡒  🡓  🡒  🡐  🡒 
 🡐  🡓  🡒  🡑  🡓 




 10%|█         | 501/5000 [01:20<14:10,  5.29it/s]

 10%|█         | 502/5000 [01:20<13:04,  5.74it/s]

 10%|█         | 503/5000 [01:20<12:25,  6.03it/s]

 10%|█         | 504/5000 [01:21<12:12,  6.14it/s]

 10%|█         | 505/5000 [01:21<11:08,  6.72it/s]

 10%|█         | 506/5000 [01:21<11:25,  6.56it/s]

 10%|█         | 508/5000 [01:21<10:42,  6.99it/s]

 10%|█         | 509/5000 [01:21<10:35,  7.06it/s]

 10%|█         | 510/5000 [01:21<11:16,  6.63it/s]

 10%|█         | 511/5000 [01:22<11:21,  6.59it/s]

 10%|█         | 512/5000 [01:22<11:23,  6.57it/s]

 10%|█         | 514/5000 [01:22<10:20,  7.23it/s]

 10%|█         | 515/5000 [01:22<11:17,  6.62it/s]

 10%|█         | 516/5000 [01:22<11:14,  6.65it/s]

 10%|█         | 517/5000 [01:22<11:36,  6.43it/s]

 10%|█         | 518/5000 [01:23<11:51,  6.30it/s]

 10%|█         | 519/5000 [01:23<12:13,  6.11it/s]

 10%|█         | 520/5000 [01:23<12:35,  5.93it/s]

 10%|█         | 521/5000 [01:23<12:21,  6.04it/s]

 10%|█    

Iteration 1000
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
 🡑  🡑  🡒  🡑  🡒 
 🡐  🡒  🡒  🡑  🡐 
 🡐  🡓  🡑  🡑  🡑 
 🡓  🡒  🡒  🡓  🡒 
 🡐  🡓  🡓  🡓  🡐 




 20%|██        | 1001/5000 [02:41<13:39,  4.88it/s]

 20%|██        | 1002/5000 [02:41<12:41,  5.25it/s]

 20%|██        | 1003/5000 [02:41<11:57,  5.57it/s]

 20%|██        | 1004/5000 [02:42<11:46,  5.66it/s]

 20%|██        | 1005/5000 [02:42<11:32,  5.77it/s]

 20%|██        | 1006/5000 [02:42<11:39,  5.71it/s]

 20%|██        | 1007/5000 [02:42<11:50,  5.62it/s]

 20%|██        | 1008/5000 [02:42<11:36,  5.73it/s]

 20%|██        | 1009/5000 [02:42<11:39,  5.71it/s]

 20%|██        | 1010/5000 [02:43<11:40,  5.69it/s]

 20%|██        | 1012/5000 [02:43<10:18,  6.45it/s]

 20%|██        | 1013/5000 [02:43<10:46,  6.17it/s]

 20%|██        | 1014/5000 [02:43<10:44,  6.19it/s]

 20%|██        | 1015/5000 [02:43<11:09,  5.95it/s]

 20%|██        | 1016/5000 [02:43<11:15,  5.90it/s]

 20%|██        | 1017/5000 [02:44<10:40,  6.21it/s]

 20%|██        | 1018/5000 [02:44<10:11,  6.51it/s]

 20%|██        | 1019/5000 [02:44<10:21,  6.40it/s]

 20%|██        | 1020/5000 [02:44<10:11,  6.

...

Iteration 4000
+---------+
|R: | : :G|
| : | : : |
| :_: : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)
 X  🡒  🡐  🡓  🡑 
 🡑  🡒  🡐  🡒  🡓 
 🡐  🡑  🡒  🡒  🡒 
 🡓  🡑  🡓  🡒  🡓 
 X  🡓  🡐  🡓  🡓 




 80%|████████  | 4001/5000 [10:42<03:13,  5.15it/s]

 80%|████████  | 4002/5000 [10:42<03:05,  5.39it/s]

 80%|████████  | 4003/5000 [10:42<02:54,  5.70it/s]

 80%|████████  | 4004/5000 [10:42<02:51,  5.82it/s]

 80%|████████  | 4005/5000 [10:42<02:42,  6.12it/s]

 80%|████████  | 4006/5000 [10:43<02:38,  6.26it/s]

 80%|████████  | 4007/5000 [10:43<02:39,  6.23it/s]

 80%|████████  | 4008/5000 [10:43<02:39,  6.22it/s]

 80%|████████  | 4009/5000 [10:43<02:42,  6.10it/s]

 80%|████████  | 4010/5000 [10:43<02:40,  6.18it/s]

 80%|████████  | 4011/5000 [10:43<02:47,  5.91it/s]

 80%|████████  | 4012/5000 [10:44<02:46,  5.93it/s]

 80%|████████  | 4013/5000 [10:44<02:41,  6.11it/s]

 80%|████████  | 4014/5000 [10:44<02:48,  5.85it/s]

 80%|████████  | 4015/5000 [10:44<02:46,  5.92it/s]

 80%|████████  | 4016/5000 [10:44<02:56,  5.59it/s]

 80%|████████  | 4017/5000 [10:45<02:54,  5.65it/s]

 80%|████████  | 4018/5000 [10:45<02:50,  5.75it/s]

 80%|████████  | 4019/5000 [10:45<02:50,  5.

Iteration 4500
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
 🡑  🡐  🡑  🡐  🡒 
 🡐  🡐  🡐  🡐  🡑 
 🡓  🡐  🡓  🡑  🡐 
 🡓  🡓  🡒  🡐  🡑 
 O  🡐  🡒  🡓  🡒 


 90%|█████████ | 4501/5000 [12:01<01:36,  5.15it/s]

 90%|█████████ | 4502/5000 [12:01<01:32,  5.38it/s]

 90%|█████████ | 4503/5000 [12:01<01:26,  5.73it/s]

 90%|█████████ | 4504/5000 [12:01<01:25,  5.77it/s]

 90%|█████████ | 4505/5000 [12:02<01:26,  5.71it/s]

 90%|█████████ | 4506/5000 [12:02<01:26,  5.71it/s]

 90%|█████████ | 4507/5000 [12:02<01:28,  5.56it/s]

 90%|█████████ | 4508/5000 [12:02<01:26,  5.67it/s]

 90%|█████████ | 4509/5000 [12:02<01:22,  5.94it/s]

 90%|█████████ | 4510/5000 [12:03<01:23,  5.85it/s]

 90%|█████████ | 4511/5000 [12:03<01:22,  5.95it/s]

 90%|█████████ | 4512/5000 [12:03<01:22,  5.90it/s]

 90%|█████████ | 4513/5000 [12:03<01:23,  5.83it/s]

 90%|█████████ | 4514/5000 [12:03<01:26,  5.65it/s]

 90%|█████████ | 4515/5000 [12:03<01:26,  5.61it/s]

 90%|█████████ | 4516/5000 [12:04<01:23,  5.78it/s]

 90%|█████████ | 4517/5000 [12:04<01:22,  5.82it/s]

 90%|█████████ | 4518/5000 [12:04<01:23,  5.79it/s]

 90%|█████████ | 4519/5000 [12:04<01:21,  5.88

Iteration 5000
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
 🡓  🡑  🡑  🡐  🡑 
 🡐  🡒  🡒  🡒  🡑 
 🡐  🡑  🡒  🡒  🡐 
 🡓  🡓  🡒  🡒  🡓 
 🡐  🡓  🡓  🡐  🡐 
{0: 0, 1: 2, 2: 4, 3: 0, 4: 1, 5: 0, 6: 3, 7: 2, 8: 1, 9: 0, 10: 0, 11: 0, 12: 3, 13: 3, 14: 3, 15: 0, 16: 5, 17: 1, 18: 3, 19: 2, 20: 0, 21: 0, 22: 0, 23: 1, 24: 1, 25: 0, 26: 3, 27: 0, 28: 3, 29: 3, 30: 0, 31: 1, 32: 1, 33: 3, 34: 1, 35: 0, 36: 3, 37: 1, 38: 0, 39: 3, 40: 0, 41: 0, 42: 3, 43: 1, 44: 1, 45: 0, 46: 0, 47: 3, 48: 1, 49: 0, 50: 0, 51: 0, 52: 3, 53: 2, 54: 1, 55: 0, 56: 0, 57: 2, 58: 1, 59: 1, 60: 0, 61: 2, 62: 0, 63: 3, 64: 3, 65: 0, 66: 1, 67: 0, 68: 1, 69: 1, 70: 0, 71: 2, 72: 1, 73: 2, 74: 0, 75: 0, 76: 0, 77: 1, 78: 1, 79: 2, 80: 0, 81: 1, 82: 3, 83: 1, 84: 3, 85: 0, 86: 4, 87: 0, 88: 3, 89: 2, 90: 0, 91: 0, 92: 2, 93: 0, 94: 1, 95: 0, 96: 2, 97: 5, 98: 0, 99: 2, 100: 0, 101: 3, 102: 1, 103: 3, 104: 0, 105: 0, 106: 0, 107: 2, 108: 2, 109: 3, 110: 0, 111: 2, 112: 2, 113: 2, 114: 0, 

In [ ]:
import matplotlib.pyplot as plt
windowed_rewards = np.convolve(rewards[-1000:], np.ones(100), 'valid')
plt.plot(windowed_rewards/100)
plt.show()

## Submission
- Submit your solution as a Jupyter notebook. 
- Ensure that all cells in the notebook have been executed and the output is showing
- Ensure that your solution consistently reaches the average cumulative reward defined in the rubric (link below)
- Post your solution on Github and share the link to your commit as a direct message in Slack

## Evaluation
The goal of the project is to get a certain average (cumulative) reward over 100 episodes. To pass the project, you must meet all the requirments in the project [rubric](https://github.com/KnowchowHQ/rl-in-action/blob/master/C1-RL-Intro/W3OH/P1-rubric.md)